### Ejemplo:

Lavado Automata para automóviles funciona sólo con un lugar. Los autos llegan siguiendo
una distribución de Poisson, con 4 autos por hora, que pueden esperar en el estacionamiento de
la instalación, si el lugar de lavado está ocupado. El tiempo para lavar y limpiar un automóvil
es exponencial, con 10 minutos de promedio. Los automóviles que no se pueden estacionar en
la instalación pueden esperar en el arrollo junto al lavado. Eso quiere decir que para todo fin
práctico no hay límite del tamaño del sistema. El gerente de la instalación desea determinar el
tamaño del estacionamiento.

Para resolver este problema, hay que determinar $L_q$, pero de momento estimemos $L$ y $W$ usando la simulación.

Calcular $L_q$ para el ejemplo anterior, usando las fórmulas de arriba.

Mis parametros para este ejercicio son:

$\lambda=4$

$\mu=6$

In [6]:
from random import expovariate
import random
from statistics import mean
from math import inf as Infinity
from random import seed
from matplotlib.pyplot import *

def l_espera(lambd,my,a):

  Num_Pkts = 100000                   #Número de paquetes a simular
  count = 0                           #Contador de paquetes simulados
  t = 0
  N = 0                               #Variable de estado: número de paquetes en el sistema

  tll = expovariate(lambd)            #Tiempo de llegada
  ts = Infinity                       #Tiempo de salida
  t_evento_anterior = 0.0             #Tiempo del último evento

  tll_Data = []                       #Lista de tiempos de llegada
  ts_Data = []                        #Lista de tiempos de salida
  r_Data = []                         #Lista de retrasos individuales
  tf_Data=[]                          #Lista de tiempos en fila
  tc_Data =[]                         #Lista de tiempos en caja

  Data = {}

  while count < Num_Pkts:
      if tll < ts:                           #Evento de llegada
          t = tll
        
#Longitud entre intervalos (tiempo)
          delta = t - t_evento_anterior
          if N in Data: 
              Data[N] += delta
          else:
              Data[N] = delta
          t_evento_anterior = t
        
          tll_Data.append(t)
          N = N + 1.0
          tll = t + expovariate(lambd) 
          if N == 1:
              tc=expovariate(my)
              ts = t + tc
              tc_Data.append(tc)
      else:                                  #Evento de salida
          t = ts
        
#Longitud entre intervalos (tiempo)
          delta = t - t_evento_anterior
          if N in Data:
              Data[N] += delta
          else:
              Data[N] = delta
          t_evento_anterior = t
        
          ts_Data.append(t)
          N = N - 1.0
          count = count + 1                         #Paquetes simulados
          if N > 0:
              tc=expovariate(my)
              ts = t + tc
              tc_Data.append(tc)
          else:
              ts = Infinity

#DATOS DE SALIDA PROCESADOS
          
#Estimación de la probabilidad de **P(N=k)**
  for (key, value) in Data.items():
      Data[key] = value/t

  media = 0.0
  media2=0.0
  medial=[]
  for (key, value) in Data.items():

        media = media + key*value
        if (key-1)>=1:
          media2= media2 + (key-1.0)*value
        medial.append(value)
                                                                                
 #Opciones por realizar:
    #1) Retraso promedio 
    #2) Número medio de clientes en el sistema 
    #3) Probabilidad de que se forme en cola  

  if a==1:
#Estimación del retraso promedio:
    for i in range(Num_Pkts):
    #for i in range(T):
        d = ts_Data[i] - tll_Data[i]                #Retraso en sistema
        d2= ts_Data[i] - tll_Data[i]-tc_Data[i]     #Retraso en fila
        tf_Data.append(d2)
        r_Data.append(d)

    print( "Retraso promedio: W = ", round( mean(r_Data), 4) )
    print("Tiempo en fila: Wq =",mean(tf_Data))

  elif a==2:
    print('E[N]: L = ', media)
    print("L_q: ",media2)

  else:
    print('¿Cuál es la probabilidad de que se forme cola? p = ', sum(Data.values())-medial[0])
    print("¿Cuál es la probabilidad de que se encuentre vacio? q = ",medial[0])

A=l_espera(4,6,2)
A=l_espera(4,6,1)

E[N]: L =  1.9864758854096252
L_q:  1.3209142890851509
Retraso promedio: W =  0.4951
Tiempo en fila: Wq = 0.3283758676187742


Encontramos nuevos parametros, gracias a nuestro código:

$L=1.9864$

$W=0.4951$

Si bien sabemos, nuestra formula de $L_q$ es:

$L_q=\lambda W_q  *-- (1)$

donde:

$W_q=W-\frac{1}{\mu}  *-- (2)$

resolviendo la ecuación (2), tenemos:

$W_q=0.4951-\frac{1}{6} = 0.3284 $  $*-- (3)$

ahora sustituimos ecuación (3) en ecuación (1):

$L_q=\lambda W_q = (4) (0.3284) = 1.3136$